<a href="https://colab.research.google.com/github/Anyaxx/Chatbot-Song-Recommender-System/blob/main/Final_chatbot_song_recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# pip install tensorflow keras pickle nltk
from google.colab import drive
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
# from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD
import random
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
intents_file = open('/content/drive/My Drive/Colab Notebooks/files required/intents.json').read()
intents = json.loads(intents_file)

In [ ]:
import nltk
nltk.download('punkt')
words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        #add documents in the corpus
        documents.append((word, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[(['hi'], 'greeting'), (['hello'], 'greeting'), (['whats', 'up'], 'greeting'), (['sup'], 'greeting'), (['is', 'anyone', 'there'], 'greeting'), (['whats', 'good'], 'greeting'), (['hey'], 'greeting'), (['bye'], 'goodbye'), (['cya'], 'goodbye'), (['see', 'you', 'later'], 'goodbye'), (['goodbye'], 'goodbye'), (['im', 'leaving'], 'goodbye'), (['have', 'a', 'good', 'day'], 'goodbye'), (['how', 'old', 'are', 'you'], 'age'), (['what', 'is', 'your', 'age'], 'age'), (['thanks'], 'thanks'), (['thank', 'you'], 'thanks'), (['thankyou'], 'thanks'), (['ty'], 'thanks'), (['I', 'owe', 'you', 'one'], 'thanks'), (['what', 'is', 'your', 'name'], 'name'), (['whats', 'your', 'name'], 'name'), (['what', 'should', 'I', 'call', 'you'], 'name'), (['how', 'should', 'I', 'address', 'you'], 'name'), (['Yes', 'it', 'does'], 'sky_net_yes'), (['Yeah'], 'sky_net_yes'), (['Haha', 'yep'], 'sky_net_yes'), (['yes'], 'sky_net_yes'), (['Indeed'], 'sky_net_yes'), (['Yup'], 'sky_net_yes'), (['Just', 'like', 'the', 'terminator

In [ ]:
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')
# create the training data
training = []
# create empty array for the output
output_empty = [0] * len(classes)
# training set, bag of words for every sentence
for doc in documents:
    # initializing bag of words

    bag = []
    # list of tokenized words for the pattern
    word_patterns = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    # create the bag of words array with 1, if word is found in current pattern
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle the features and make numpy array
random.shuffle(training)
training = np.array(training)
# create training and testing lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data is created")

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


Training data is created


<ipython-input-4-4654f6fe2634>:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [ ]:
# deep neural networds model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compiling model. SGD with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#Training and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model is created")

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
23/23 [==============================] - 1s 4ms/step - loss: 3.1279 - accuracy: 0.0541
Epoch 2/200
23/23 [==============================] - 0s 3ms/step - loss: 3.0174 - accuracy: 0.1261
Epoch 3/200
23/23 [==============================] - 0s 2ms/step - loss: 2.9473 - accuracy: 0.1712
Epoch 4/200
23/23 [==============================] - 0s 2ms/step - loss: 2.7867 - accuracy: 0.2072
Epoch 5/200
23/23 [==============================] - 0s 2ms/step - loss: 2.6880 - accuracy: 0.2703
Epoch 6/200
23/23 [==============================] - 0s 3ms/step - loss: 2.6190 - accuracy: 0.2162
Epoch 7/200
23/23 [==============================] - 0s 3ms/step - loss: 2.5410 - accuracy: 0.2252
Epoch 8/200
23/23 [==============================] - 0s 2ms/step - loss: 2.3863 - accuracy: 0.3153
Epoch 9/200
23/23 [==============================] - 0s 2ms/step - loss: 2.3540 - accuracy: 0.3243
Epoch 10/200
23/23 [==============================] - 0s 2ms/step - loss: 2.2381 - accuracy: 0.3514
Epoch 11/

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('/content/drive/MyDrive/Colab Notebooks/files required/intents.json').read())
words = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/files required/words.pkl','rb'))
classes = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/files required/classes.pkl','rb'))
def clean_up_sentence(sentence):
    # tokenize the pattern - splitting words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stemming every word - reducing to base form
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for words that exist in sentence
def bag_of_words(sentence, words, show_details=True):
    # tokenizing patterns
    sentence_words = clean_up_sentence(sentence)
    # bag of words - vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,word in enumerate(words):
            if word == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % word)
    return(np.array(bag))
def predict_class(sentence):
    # filter below  threshold predictions
    p = bag_of_words(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sorting strength probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
#Creating tkinter GUI
# import tkinter
# from tkinter import *
# def send():
#     msg = EntryBox.get("1.0",'end-1c').strip()
#     EntryBox.delete("0.0",END)
#     if msg != '':
#         ChatBox.config(state=NORMAL)
#         ChatBox.insert(END, "You: " + msg + '\n\n')
#         ChatBox.config(foreground="#446665", font=("Verdana", 12 )) 
#         ints = predict_class(msg)
#         res = getResponse(ints, intents)
#         ChatBox.insert(END, "Bot: " + res + '\n\n')           
#         ChatBox.config(state=DISABLED)
#         ChatBox.yview(END)
# root = Tk()
# root.title("Chatbot")
# root.geometry("400x500")
# root.resizable(width=FALSE, height=FALSE)
# #Create Chat window
# ChatBox = Text(root, bd=0, bg="white", height="8", width="50", font="Arial",)
# ChatBox.config(state=DISABLED)
# #Bind scrollbar to Chat window
# scrollbar = Scrollbar(root, command=ChatBox.yview, cursor="heart")
# ChatBox['yscrollcommand'] = scrollbar.set
# #Create Button to send message
# SendButton = Button(root, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
#                     bd=0, bg="#f9a602", activebackground="#3c9d9b",fg='#000000',
#                     command= send )
# #Create the box to enter message
# EntryBox = Text(root, bd=0, bg="white",width="29", height="5", font="Arial")
# #EntryBox.bind("<Return>", send)
# #Place all components on the screen
# scrollbar.place(x=376,y=6, height=386)
# ChatBox.place(x=6,y=6, height=386, width=370)
# EntryBox.place(x=128, y=401, height=90, width=265)
# SendButton.place(x=6, y=401, height=90)
# root.mainloop()

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
# from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD
import random

import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']
intents_file = open('/content/drive/MyDrive/Colab Notebooks/files required/intents.json').read()
intents = json.loads(intents_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        #add documents in the corpus
        documents.append((word, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for word in words:
        bag.append(1) if word in pattern_words else bag.append(0)
        
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=172, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

[(['hi'], 'greeting'), (['hello'], 'greeting'), (['whats', 'up'], 'greeting'), (['sup'], 'greeting'), (['is', 'anyone', 'there'], 'greeting'), (['whats', 'good'], 'greeting'), (['hey'], 'greeting'), (['bye'], 'goodbye'), (['cya'], 'goodbye'), (['see', 'you', 'later'], 'goodbye'), (['goodbye'], 'goodbye'), (['im', 'leaving'], 'goodbye'), (['have', 'a', 'good', 'day'], 'goodbye'), (['how', 'old', 'are', 'you'], 'age'), (['what', 'is', 'your', 'age'], 'age'), (['thanks'], 'thanks'), (['thank', 'you'], 'thanks'), (['thankyou'], 'thanks'), (['ty'], 'thanks'), (['I', 'owe', 'you', 'one'], 'thanks'), (['what', 'is', 'your', 'name'], 'name'), (['whats', 'your', 'name'], 'name'), (['what', 'should', 'I', 'call', 'you'], 'name'), (['how', 'should', 'I', 'address', 'you'], 'name'), (['Yes', 'it', 'does'], 'sky_net_yes'), (['Yeah'], 'sky_net_yes'), (['Haha', 'yep'], 'sky_net_yes'), (['yes'], 'sky_net_yes'), (['Indeed'], 'sky_net_yes'), (['Yup'], 'sky_net_yes'), (['Just', 'like', 'the', 'terminator

<ipython-input-7-37b2c209c743>:70: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


23/23 [==============================] - 1s 2ms/step - loss: 3.0647 - accuracy: 0.0541 
Epoch 2/172
23/23 [==============================] - 0s 3ms/step - loss: 2.9906 - accuracy: 0.1171
Epoch 3/172
23/23 [==============================] - 0s 3ms/step - loss: 2.9401 - accuracy: 0.1081
Epoch 4/172
23/23 [==============================] - 0s 2ms/step - loss: 2.8272 - accuracy: 0.1712
Epoch 5/172
23/23 [==============================] - 0s 3ms/step - loss: 2.7498 - accuracy: 0.2252
Epoch 6/172
23/23 [==============================] - 0s 2ms/step - loss: 2.7247 - accuracy: 0.2072
Epoch 7/172
23/23 [==============================] - 0s 2ms/step - loss: 2.6467 - accuracy: 0.2523
Epoch 8/172
23/23 [==============================] - 0s 2ms/step - loss: 2.6053 - accuracy: 0.2432
Epoch 9/172
23/23 [==============================] - 0s 3ms/step - loss: 2.4312 - accuracy: 0.2703
Epoch 10/172
23/23 [==============================] - 0s 3ms/step - loss: 2.3187 - accuracy: 0.2703
Epoch 11/172
23/23 [

In [ ]:
!pip install --upgrade "ibm-watson>=6.0.0"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 KB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 7.5 MB/s eta 0:00:00
  Created wheel for ibm-watson: filename=ibm_watson-6.1.0-py3-none-any.whl size=370764 sha256=ccf47bf8d3a67e75a31bb0085846b192fb1d663df7d2fa3b06e371a058c9db91
  Stored in directory: /root/.cache/pip/wheels/34/b4/cd/829a351c802b7a

In [ ]:
msg = list()
text = str()

In [ ]:
# import json
# from ibm_watson import NaturalLanguageUnderstandingV1
# from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
# from ibm_watson.natural_language_understanding_v1 \
#     import Features, EmotionOptions

# authenticator = IAMAuthenticator('WwUGO6YkM_-sv8caNXoPiZI9Zp7V01_E_HCSYIJ0SSJN')
# natural_language_understanding = NaturalLanguageUnderstandingV1(
#     version='2022-04-07',
#     authenticator=authenticator
# )

# natural_language_understanding.set_service_url('https://api.au-syd.natural-language-understanding.watson.cloud.ibm.com/instances/33671fd1-d455-441f-a82a-9c01a524b0b0')

# response = natural_language_understanding.analyze(
#     text="I'm so happy today",
#     features=Features(emotion=EmotionOptions())).get_result()

# # print(json.dumps(response, indent=2))
# emotions_dict = response["emotion"]["document"]["emotion"]
# # print(emotions_dict)
# final_emotion = max(emotions_dict, key= lambda x: emotions_dict[x])
# print(final_emotion)

In [ ]:
def responsed(msg1):
    msg.append(msg1)
    ints = predict_class(msg1)
    res = getResponse(ints, intents)
    return res


In [ ]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 \
    import Features, EmotionOptions

def song_emotion():
    authenticator = IAMAuthenticator('WwUGO6YkM_-sv8caNXoPiZI9Zp7V01_E_HCSYIJ0SSJN')
    natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2022-04-07',
    authenticator=authenticator
)

    natural_language_understanding.set_service_url('https://api.au-syd.natural-language-understanding.watson.cloud.ibm.com/instances/33671fd1-d455-441f-a82a-9c01a524b0b0')
    # text = ""
    # for i in msg:
    #     text = text+i
    len1 = len(msg)
    nlu = natural_language_understanding.analyze(
        text = msg[len1-1]+" "+msg[len1-2]+" "+msg[len1-3]+" "+msg[len1-4]+" "+msg[len1-5],
        features=Features(emotion=EmotionOptions())).get_result()
    
    # print(json.dumps(nlu, indent=2))
    emotions_dict = nlu["emotion"]["document"]["emotion"]
    # print(emotions_dict)
    final_emotion = max(emotions_dict, key= lambda x: emotions_dict[x])
    # print(final_emotion)
    emotion_songs = dict()
    emotion_songs['emotion'] = final_emotion

    import requests

    url=f"http://ws.audioscrobbler.com/2.0/?method=tag.gettoptracks&tag={final_emotion}&api_key=419a54c976669b85ed555332a0fada25&format=json&limit=10"
    response = requests.get(url)
    payload = response.json()
    for i in range(10):
        r=payload['tracks']['track'][i]
        emotion_songs[r['name']] = r['url']
    return emotion_songs

In [ ]:
# SONG RECOMMENDATION

In [ ]:
print("Chatbot : Hey there, Wassup?")
# responded function takes text of user and returns chatbot output
for i in range(5):
    m = input("User : ")
    res = responsed(m)
    print("Chatbot : "+res)
ans = song_emotion()
print("Emotion : "+ans['emotion'])

Chatbot : Hey there, Wassup?
User : hello
1/1 [==============================] - 0s 186ms/step
Chatbot : Hello
User : hi
1/1 [==============================] - 0s 90ms/step
Chatbot : Hey there
User : how are you
1/1 [==============================] - 0s 21ms/step
Chatbot : good, wby?
User : good
1/1 [==============================] - 0s 19ms/step
Chatbot : Thanks. I have been told before that I am quite the comedian.
User : tell me a joke
1/1 [==============================] - 0s 16ms/step
Chatbot : A guy goes into a lawyer’s office and asks the lawyer: Excuse me, how much do you charge?
The lawyer responds: I charge £1,000 to answer three questions.
Bloody hell – That’s a bit expensive isn’t it?
Yes. What’s your third question?
Emotion : joy


In [ ]:
# song_emotion function would return dictionary consisting of emotion and recommended songs
ans = song_emotion()
print("Emotion : "+ans['emotion'])
ans.pop('emotion')
lst = list(ans.keys())
print("Song Recommendations : ")
for i in range(10):
    print("Song_name : "+lst[i])
    print("Song_URL : "+ans[lst[i]])


Emotion : joy
Song Recommendations : 
Song_name : You And Your Heart
Song_URL : https://www.last.fm/music/Jack+Johnson/_/You+And+Your+Heart
Song_name : praise you - radio edit
Song_URL : https://www.last.fm/music/Fatboy+Slim/_/praise+you+-+radio+edit
Song_name : Yoshimi Battles The Pink Robots Part 1
Song_URL : https://www.last.fm/music/The+Flaming+Lips/_/Yoshimi+Battles+The+Pink+Robots+Part+1
Song_name : Come on! Feel the Illinoise!
Song_URL : https://www.last.fm/music/Sufjan+Stevens/_/Come+on%21+Feel+the+Illinoise%21
Song_name : Let's Go
Song_URL : https://www.last.fm/music/Matt+&+Kim/_/Let%27s+Go
Song_name : Lake Shore Drive
Song_URL : https://www.last.fm/music/Aliotta+Haynes+Jeremiah/_/Lake+Shore+Drive
Song_name : Old Old Fashioned
Song_URL : https://www.last.fm/music/Frightened+Rabbit/_/Old+Old+Fashioned
Song_name : Hello
Song_URL : https://www.last.fm/music/Joy/_/Hello
Song_name : Dontcha
Song_URL : https://www.last.fm/music/The+Internet/_/Dontcha
Song_name : My Little Town
Song_